## Load Environment & Detectron 2


In [ ]:
!git clone https://github.com/airsplay/py-bottom-up-attention

In [ ]:
import os
os.chdir(f'./py-bottom-up-attention')

!pip install -r requirements.txt
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!python setup.py build develop

In [ ]:
import os
import io

import detectron2

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# import some common libraries
import numpy as np
import cv2
import torch

# Show the image in ipynb
from IPython.display import clear_output, Image, display
import PIL.Image
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [ ]:
# Load VG Classes
data_path = '/content/py-bottom-up-attention/demo/data/genome/1600-400-20'

vg_classes = []
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        vg_classes.append(object.split(',')[0].lower().strip())

MetadataCatalog.get("vg").thing_classes = vg_classes

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/content/py-bottom-up-attention/configs/VG-Detection/faster_rcnn_R_101_C4_caffe.yaml")
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 300
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# VG Weight
cfg.MODEL.WEIGHTS = "http://nlp.cs.unc.edu/models/faster_rcnn_from_caffe.pkl"
predictor = DefaultPredictor(cfg)

## Load Pic

### Sample Pic

In [ ]:
im = cv2.imread("/content/py-bottom-up-attention/demo/data/images/input.jpg")
im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
showarray(im_rgb)

### Our dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
im = cv2.imread("/content/drive/MyDrive/Capstone/Labeling/dataset/Bar_graph/one_bar_plots/Bar0.png")
im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
showarray(im_rgb)


## Use detectron 2 to extract features 

### Sample pic

In [ ]:
NUM_OBJECTS = 36

from torch import nn

from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances

def doit(raw_image, raw_boxes):
        # Process Boxes
    raw_boxes = Boxes(torch.from_numpy(raw_boxes).cuda())
    
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        print("Transformed image size: ", image.shape[:2])
        
        # Scale the box
        new_height, new_width = image.shape[:2]
        scale_x = 1. * new_width / raw_width
        scale_y = 1. * new_height / raw_height
        #print(scale_x, scale_y)
        boxes = raw_boxes.clone()
        boxes.scale(scale_x=scale_x, scale_y=scale_y)
        
        # ----
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [boxes]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        print(pred_class_logits.shape)
        pred_class_prob = nn.functional.softmax(pred_class_logits, -1)
        pred_scores, pred_classes = pred_class_prob[..., :-1].max(-1)
        
        # Detectron2 Formatting (for visualization only)
        roi_features = feature_pooled
        instances = Instances(
            image_size=(raw_height, raw_width),
            pred_boxes=raw_boxes,
            scores=pred_scores,
            pred_classes=pred_classes
        )
        
        return instances, roi_features
    
given_boxes = np.array(
    [[1.7333e+02, 2.1515e+02, 4.8672e+02, 4.7373e+02],
        [1.2166e+02, 2.0614e+02, 3.4905e+02, 4.8000e+02],
        [5.8896e+02, 0.0000e+00, 6.3909e+02, 3.6998e+02],
        [6.0792e+02, 9.0849e+01, 6.3765e+02, 4.2150e+02],
        [2.8171e+02, 1.6275e+02, 3.2904e+02, 1.9557e+02],
        [1.5337e+02, 9.6636e+01, 3.9307e+02, 4.5865e+02],
        [3.9510e+00, 3.0141e-01, 1.7087e+02, 3.7003e+02],
        [2.0478e+02, 0.0000e+00, 3.0078e+02, 2.7645e+02],
        [3.8164e+02, 3.1898e+02, 6.1028e+02, 4.2289e+02],
        [4.2380e+02, 2.7979e+02, 6.3800e+02, 3.9043e+02],
        [5.3907e+01, 2.1506e+01, 1.2955e+02, 3.8665e+02],
        [2.1639e+02, 3.3180e+02, 4.9085e+02, 4.7821e+02],
        [4.5419e+01, 3.1766e+02, 5.8115e+02, 4.7680e+02],
        [5.2262e+01, 1.5123e+02, 4.9093e+02, 4.3199e+02],
        [3.4266e+02, 4.8674e+01, 6.3398e+02, 3.8901e+02],
        [2.4584e+02, 1.8033e+02, 3.4975e+02, 4.0818e+02],
        [1.7189e+02, 1.6335e+02, 6.3919e+02, 4.1045e+02],
        [1.9629e+01, 0.0000e+00, 5.6497e+02, 1.5761e+02],
        [3.9222e+02, 0.0000e+00, 6.3402e+02, 2.7783e+02],
        [3.6025e+01, 0.0000e+00, 5.5431e+02, 2.8221e+02],
        [1.5994e+02, 1.5115e+00, 3.5376e+02, 3.1772e+02],
        [2.9326e+02, 1.4786e+02, 3.2540e+02, 1.8938e+02],
        [0.0000e+00, 3.6491e+02, 4.3185e+02, 4.7849e+02],
        [1.9907e+01, 4.2409e+02, 4.5854e+02, 4.7957e+02],
        [4.9555e+00, 8.1566e+01, 2.3710e+02, 4.5235e+02],
        [5.5625e+02, 2.7353e+02, 6.0216e+02, 3.7322e+02],
        [9.2086e+01, 2.8328e+02, 3.2548e+02, 4.4708e+02],
        [1.7761e+02, 3.6624e+02, 4.5720e+02, 4.6956e+02],
        [1.7253e+02, 3.7161e+02, 6.4000e+02, 4.7876e+02],
        [2.7954e+02, 2.0651e+02, 3.4036e+02, 3.1626e+02],
        [1.9732e+02, 3.8317e+01, 6.4000e+02, 3.2455e+02],
        [2.7082e+02, 1.1922e+00, 5.8803e+02, 3.0338e+02],
        [6.5850e+00, 1.8703e+02, 3.0191e+02, 4.7954e+02],
        [0.0000e+00, 0.0000e+00, 2.2748e+02, 2.3305e+02],
        [2.4732e-01, 3.3860e+02, 3.1494e+02, 4.7737e+02],
        [2.0554e+02, 1.9619e+00, 6.4000e+02, 2.7667e+02]]
)
instances, features = doit(im, given_boxes)

print("Classes", instances.pred_classes)

In [ ]:
# Show the boxes, labels, and features
pred = instances.to('cpu')
v = Visualizer(im[:, :, :], MetadataCatalog.get("vg"), scale=1.2)
v = v.draw_instance_predictions(pred)
showarray(v.get_image()[:, :, ::-1])
print('instances:\n', instances)
print()
print('boxes:\n', instances.pred_boxes)
print()
print('Shape of features:\n', features.shape)

### Our bar chart 0

In [ ]:
# NUM_OBJECTS = 5

from torch import nn

from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances

def doit(raw_image, raw_boxes):
        # Process Boxes
    raw_boxes = Boxes(torch.from_numpy(raw_boxes).cuda())
    
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        print("Transformed image size: ", image.shape[:2])
        
        # Scale the box
        new_height, new_width = image.shape[:2]
        scale_x = 1. * new_width / raw_width
        scale_y = 1. * new_height / raw_height
        print(scale_x, scale_y)
        boxes = raw_boxes.clone()
        boxes.scale(scale_x=scale_x, scale_y=scale_y)
        
        # ----
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [boxes]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        print(pred_class_logits.shape)
        pred_class_prob = nn.functional.softmax(pred_class_logits, -1)
        pred_scores, pred_classes = pred_class_prob[..., :-1].max(-1)
        
        # Detectron2 Formatting (for visualization only)
        roi_features = feature_pooled
        instances = Instances(
            image_size=(raw_height, raw_width),
            pred_boxes=raw_boxes,
            scores=pred_scores,
            pred_classes=pred_classes
        )
        
        return instances, roi_features
    


In [ ]:
given_boxes = np.array(
    [[282.575757575758,424.061426956084,503.068036688726,1444.44444444444],
    [912.553697898525,342.894619894955,1133.04597701149,1444.44444444444],
    [1542.53163822129,296.513618883178,1763.02391733426,1444.44444444444],
    [2172.50957854406,203.751506278598,2393.00185765703,1444.44444444444],
    [2802.48751886683,134.179894179894,3022.9797979798,1444.44444444444]]
)

instances, features = doit(im, given_boxes)

print("Classes", instances.pred_classes)

In [ ]:
features

In [ ]:
# Show the boxes, labels, and features

pred = instances.to('cpu')
v = Visualizer(im[:, :, :], MetadataCatalog.get("vg"), scale=1.2)
v = v.draw_instance_predictions(pred)
showarray(v.get_image()[:, :, ::-1])


print('instances:\n', instances)
print()
print('boxes:\n', instances.pred_boxes)
print()
print('Shape of features:\n', features.shape)

## Load bar_box_one_bar coordinates dict

In [ ]:
import json

with open('/content/drive/MyDrive/Capstone/Labeling/JSON/dict_bar1_box_all_one_bar_calculated.json') as f:
    dict_bar1_box_all_one_bar_calculated = json.load(f) # len = 1592

In [ ]:
len(dict_bar1_box_all_one_bar_calculated)

In [ ]:
dict_bar1_box_all_one_bar_calculated.keys()

## Construct function to return all one bar features

In [ ]:
#test on a different image

img_id = 2222
chart_file_name = 'Bar'+str(img_id)+'.png'
im = cv2.imread("/content/drive/MyDrive/Capstone/Labeling/dataset/Bar_graph/one_bar_plots/"+chart_file_name)
given_boxes_test = np.array(list(dict_bar1_box_all_one_bar_calculated[str(img_id)].values()))
instances, features = doit(im, given_boxes_test)

# Show the boxes, labels, and features
pred = instances.to('cpu')
v = Visualizer(im[:, :, :], MetadataCatalog.get("vg"), scale=1.2)
v = v.draw_instance_predictions(pred)
showarray(v.get_image()[:, :, ::-1])
print('instances:\n', instances)
print()
print('boxes:\n', instances.pred_boxes)
print()
print('Shape of features:\n', features.shape)

### Same code without prints

In [ ]:
from torch import nn

from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances

def doit(raw_image, raw_boxes):
        # Process Boxes
    raw_boxes = Boxes(torch.from_numpy(raw_boxes).cuda())
    
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        #print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        #print("Transformed image size: ", image.shape[:2])
        
        # Scale the box
        new_height, new_width = image.shape[:2]
        scale_x = 1. * new_width / raw_width
        scale_y = 1. * new_height / raw_height
        #print(scale_x, scale_y)
        boxes = raw_boxes.clone()
        boxes.scale(scale_x=scale_x, scale_y=scale_y)
        
        # ----
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [boxes]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        #print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        #print(pred_class_logits.shape)
        pred_class_prob = nn.functional.softmax(pred_class_logits, -1)
        pred_scores, pred_classes = pred_class_prob[..., :-1].max(-1)
        
        # Detectron2 Formatting (for visualization only)
        roi_features = feature_pooled
        instances = Instances(
            image_size=(raw_height, raw_width),
            pred_boxes=raw_boxes,
            scores=pred_scores,
            pred_classes=pred_classes
        )
        
        return instances, roi_features

In [ ]:
one_bar_img_features={}

for img_id in dict_bar1_box_all_one_bar_calculated.keys():
  chart_file_name = 'Bar'+img_id+'.png'
  im = cv2.imread("/content/drive/MyDrive/Capstone/Labeling/dataset/Bar_graph/one_bar_plots/"+chart_file_name)
  im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  
  given_boxes = np.array(list(dict_bar1_box_all_one_bar_calculated[img_id].values()))
  instances, features = doit(im, given_boxes_test)
  one_bar_img_features[img_id] = features


In [ ]:
len(one_bar_img_features)

In [ ]:
one_bar_img_features['0']